In [ ]:
import pandas as pd
import numpy as np
import os
import time
import matplotlib.pyplot as plt

In [ ]:
# GA Parameters
POPULATION_SIZE = 100 
GENES_LENGTH = 100 # must be adjusted according to the instance
MUTATION_RATE = 0.4
CROSSOVER_RATE = 0.8
TOURNAMENT_SIZE = 3
OSX_SEGMENT_LENGTH = int(GENES_LENGTH // 3)
OPX_SEGMENT_LENGTH = int(GENES_LENGTH // 3)
TPX_SEGMENT_LENGTH = int(GENES_LENGTH // 3)
MUTATION_SCRAMBLE_LENGTH = int(GENES_LENGTH // 3)
MUTATION_INVERSION_LENGTH = int(GENES_LENGTH // 3)
MAX_ELITISM_PERCENTAGE = 0.25
MIN_ELITISM_PERCENTAGE = 0.05
ADJUSTMENT_INTERVAL = 10
LOCAL_SEARCH_RATE = 0.0
LOCAL_SEARCH_MOVES = 2500
GENERATIONS = 600
RESTART_INTERVAL = 50
MAX_RESTART_ATTEMPTS = 10

MUTATION_TYPE = "flip"
CROSSOVER_TYPE = "TPX"
SELECTION_TYPE = "roulette"

TAU = 1



# Free space at the two stages
D1 = 15
D2 = 11000

# Print all output
DUMMY = False
EVOLUTION_PLOT_INTERVAL = 10

In [ ]:
class Instance:
    def __init__(self, x):
        """
        Initializes the instance with default values and loads job data from a file.

        Args:
        x (str): The file name to load job data from.

        Returns:
        None
        """
        self.J = 0
        self.F = 0
        self.m1 = 0
        self.m2 = 0
        self.k1 = 0
        self.k2 = 0

        self.file = x
        self.filename = FOLDER_PATH + x
        print(f"Loading instance from file: {x}")
        self.read_jobs_from_file()
        self.lower_bound = self.calculate_lower_bound()
        self.jobs = self.sules_rule(self.jobs)
        self.SPT_jobs = self.calculate_SPT_jobs()
        self.ERD_jobs = self.calculate_ERD_jobs()
        self.LPT_jobs = self.calculate_LPT_jobs()
        self.TIME_LIMIT = self.calculate_time_limit()
        if DUMMY:
            print(f"Time limit: {self.TIME_LIMIT}")

    def calculate_LPT_jobs(self):
        jobs = self.jobs.copy()  # Create a copy of self.instance.jobs
        jobs = jobs[jobs[:, 1].argsort()[::-1]] - 1

        tmp = list(jobs[:, 0])
        return tmp

    def calculate_time_limit(self):
        return min(((5000+TAU*(self.J**2)*((self.m1+self.m2)/2))/1000), 1800)

    def calculate_SPT_jobs(self):
        jobs = self.jobs.copy()  # Create a copy of self.instance.jobs
        jobs = jobs[jobs[:, 1].argsort()] - 1
        tmp = list(jobs[:, 0])
        return tmp

    def calculate_ERD_jobs(self):
        jobs = self.jobs.copy()  # Create a copy of self.instance.jobs
        jobs = jobs[jobs[:, 3].argsort()] - 1
        tmp = list(jobs[:, 0])
        return tmp

    def read_jobs_from_file(self):
        """
            Reads job data from a file.

            Args:
            filename (str): Name of the file to read from.

            Returns:
            tuple: Parameters of the job data and the job data itself.
            """
        try:
            with open(self.filename, 'r') as file:
                self.J, self.F, self.m1, self.m2, self.k1, self.k2 = map(int, file.readline().split())
                self.jobs = np.zeros((self.J, 9))
                for i in range(self.J):
                    job_info = list(map(float, file.readline().split()))
                    self.jobs[i] = job_info
            if DUMMY:
                print(
                    f"Instance loaded wit J = {self.J}, F = {self.F}, m1 = {self.m1}, m2 = {self.m2}, k1 = {self.k1}, k2 = {self.k2}.")
                print("")
        except IOError as e:
            print(f"Error reading from file: {e}")
            return None

    def sules_rule(self, jobs):
        """
        Applies Sule's rule to jobs, updating the processing time.

        Args:
        jobs (numpy.ndarray): Array of job data.

        Returns:
        numpy.ndarray: Modified array of job data after applying Sule's rule.
        """
        jobs_sules_rule = jobs.copy()
        jobs_sules_rule[:, 1] += jobs_sules_rule[:, 5]
        jobs_sules_rule[:, 2] += jobs_sules_rule[:, 6]
        jobs_sules_rule = np.delete(jobs_sules_rule, [5, 6], axis=1)
        return jobs_sules_rule

    def calculate_lower_bound(self):
        # Assuming jobs is a 2D array where:
        # Column 1 is processing time for stage 1, Column 2 is processing time for stage 2,
        # Column 3 is release time, Column 5 is setup time for stage 1, Column 6 is setup time for stage 2.
        processing_time_sum = np.sum(self.jobs[:, 1]) + np.sum(self.jobs[:, 2])
        release_time_sum = np.sum(self.jobs[:, 3])
        setup_time_sum = np.sum(self.jobs[:, 5]) + np.sum(self.jobs[:, 6])

        lower_bound = processin

In [ ]:
class Genetic_Algorithm:
    def __init__(self, instance):
        self.instance = instance
        self.solution_time = 0
        self.fitness_time = 0
        self.selection_time = 0
        self.crossover_time = 0
        self.mutation_time = 0
        self.local_search_time = 0
        self.restart_time = 0
        self.population = self.generate_population()
        self.population.append(self.instance.SPT_jobs)
        self.population.append(self.instance.ERD_jobs)
        self.population.append(self.instance.LPT_jobs)
        self.elite = []
        self.fitness_score = []
        self.elitism_percentage = MAX_ELITISM_PERCENTAGE
        self.elite_size = int(self.elitism_percentage * POPULATION_SIZE)
        self.best_fitness_values = []
        self.restart_interval = RESTART_INTERVAL
        self.max_restart_attempts = MAX_RESTART_ATTEMPTS
        self.current_restart_attempt = 0

    def convergence_analysis(self):
        generations = range(len(self.best_fitness_values))
        plt.plot(generations, self.best_fitness_values)
        plt.xlabel('Generation')
        plt.ylabel('Best Fitness Value')
        plt.title('Convergence Analysis')
        plt.grid(True)
        plt.show()

    def sensitivity_analysis(self, parameter_values, performance_metrics):
        plt.plot(parameter_values, performance_metrics)
        plt.xlabel('Parameter Value')
        plt.ylabel('Performance Metric')
        plt.title('Sensitivity Analysis')
        plt.grid(True)
        plt.show()

    def generate_population(self):
        return [np.random.permutation(np.arange(0, GENES_LENGTH, 1)) for _ in range(POPULATION_SIZE - 3)]

    def fitness(self, individual):
        jobs_rescaled = np.copy(self.instance.jobs)
        index_classified_family = self.index_by_family(jobs_rescaled, individual)
        jobs_classified = self.classify_jobs(jobs_rescaled, index_classified_family)

        # Process Stage 1
        Batches, Processing_times, Release_times = self.ordered_batches(jobs_classified, self.instance.F,
                                                                        self.instance.k1, 1, D1)
        mProcessing_times = self.operating_times_batches(Batches, Processing_times)
        mRelease_times = self.operating_times_batches(Batches, Release_times)
        scheduled_batches, completion_times_batches = self.schedule_batches_spt(mRelease_times, mProcessing_times,
                                                                                self.instance.m1, self.instance.F)

        # Calculate completion and starting times for Stage 1
        completion_times_stage_1, starting_times, machine_ids = self.calculate_times_for_stage(Batches,
                                                                                               scheduled_batches,
                                                                                               GENES_LENGTH)

        # Process Stage 2
        jobs_stage_2_rescaled = self.overwrite_release_times(jobs_rescaled, completion_times_stage_1)
        index_classified_family_stage_2 = self.index_by_family(jobs_stage_2_rescaled, individual)
        jobs_classified_stage_2 = self.classify_jobs(jobs_stage_2_rescaled, index_classified_family_stage_2)
        Batches_stage_2, Processing_times_stage_2, Release_times_stage_2 = self.ordered_batches(jobs_classified_stage_2,
                                                                                                self.instance.F,
                                                                                                self.instance.k2, 2, D2)
        mProcessing_times_stage_2 = self.operating_times_batches(Batches_stage_2, Processing_times_stage_2)
        mRelease_times_stage_2 = self.operating_times_batches(Batches_stage_2, Release_times_stage_2)
        scheduled_batches_stage_2, completion_times_batches_stage_2 = self.schedule_batches_spt(mRelease_times_stage_2,
                                                                                                mProcessing_times_stage_2,
                                                                                                self.instance.m2,
                                                                                                self.instance.F)

        # Calculate completion and starting times for Stage 2
        completion_times_stage_2, starting_times_stage_2, machine_ids_stage_2 = self.calculate_times_for_stage(
            Batches_stage_2, scheduled_batches_stage_2, GENES_LENGTH)

        # Calculate heuristic result and lower bound
        return sum(completion_times_stage_2)

    def print_individual(self, individual):
        jobs_rescaled = self.instance.jobs.copy()
        index_classified_family = self.index_by_family(jobs_rescaled, individual)
        jobs_classified = self.classify_jobs(jobs_rescaled, index_classified_family)

        # Process Stage 1
        Batches, Processing_times, Release_times = self.ordered_batches(jobs_classified, self.instance.F,
                                                                        self.instance.k1, 1, D1)
        mProcessing_times = self.operating_times_batches(Batches, Processing_times)
        mRelease_times = self.operating_times_batches(Batches, Release_times)
        scheduled_batches, completion_times_batches = self.schedule_batches_spt(mRelease_times, mProcessing_times,
                                                                                self.instance.m1, self.instance.F)

        # Calculate completion and starting times for Stage 1
        completion_times_stage_1, starting_times, machine_ids = self.calculate_times_for_stage(Batches,
                                                                                               scheduled_batches,
                                                                                               GENES_LENGTH)

        # Process Stage 2
        jobs_stage_2_rescaled = self.overwrite_release_times(jobs_rescaled, completion_times_stage_1)
        index_classified_family_stage_2 = self.index_by_family(jobs_stage_2_rescaled, individual)
        jobs_classified_stage_2 = self.classify_jobs(jobs_stage_2_rescaled, index_classified_family_stage_2)
        Batches_stage_2, Processing_times_stage_2, Release_times_stage_2 = self.ordered_batches(jobs_classified_stage_2,
                                                                                                self.instance.F,
                                                                                                self.instance.k2, 2, D2)
        mProcessing_times_stage_2 = self.operating_times_batches(Batches_stage_2, Processing_times_stage_2)
        mRelease_times_stage_2 = self.operating_times_batches(Batches_stage_2, Release_times_stage_2)
        scheduled_batches_stage_2, completion_times_batches_stage_2 = self.schedule_batches_spt(mRelease_times_stage_2,
                                                                                                mProcessing_times_stage_2,
                                                                                                self.instance.m2,
                                                                                                self.instance.F)

        # Calculate completion and starting times for Stage 2
        completion_times_stage_2, starting_times_stage_2, machine_ids_stage_2 = self.calculate_times_for_stage(
            Batches_stage_2, scheduled_batches_stage_2, GENES_LENGTH)
        file = f"GA_{self.instance.file}"
        self.write_solution_file(file, scheduled_batches, scheduled_batches_stage_2, starting_times,
                                     completion_times_stage_1, machine_ids, starting_times_stage_2,
                                     completion_times_stage_2,
                                     machine_ids_stage_2, self.instance.m1, self.instance.m2, self.instance.lower_bound)
        self.write_detailed_solution(file, scheduled_batches, scheduled_batches_stage_2, starting_times,
                                     completion_times_stage_1, machine_ids, starting_times_stage_2,
                                     completion_times_stage_2,
                                     machine_ids_stage_2, self.instance.m1, self.instance.m2, self.instance.lower_bound)

    def index_by_family(self, jobs, individual):
        indexed_jobs = [[] for _ in range(self.instance.F)]
        for j in individual:
            f = int(jobs[int(j), 4])
            indexed_jobs[f - 1].append(int(j))
        return indexed_jobs

    def classify_jobs(self, jobs, indexed_jobs):
        """
        Classifies jobs by their family.

        Args:
        jobs (numpy.ndarray): Array of job data.
        indexed_jobs (list of list): Indices of jobs classified by family.

        Returns:
        list of numpy.ndarray: Jobs classified by family.
        """
        jobs_classified = []
        for family_indices in indexed_jobs:
            jobs_f = jobs[family_indices]
            jobs_classified.append(jobs_f)
        return jobs_classified

    def create_batch_space_limit(self, remaining_jobs, batch_size, stage, delta):
        """
        Creates a batch of jobs considering different job sizes and a space limit.

        Args:
        remaining_jobs (numpy.ndarray): Array of remaining jobs.
        batch_size (int): Maximum size of the batch.
        stage (int): The stage for which the batch is being created.
        delta (int): The space limit for the batch.

        Returns:
        tuple: A tuple containing the batch queue, updated remaining jobs,
               maximum processing time, and maximum release time in the batch.
        """
        batch_queue = []
        processing_time_q = 0
        release_time_q = 0
        current_batch_size = 0
        while current_batch_size < delta and len(remaining_jobs) >= 1:
            j = len(remaining_jobs) - 1
            job = remaining_jobs[j]
            if current_batch_size + job[4 + stage] > batch_size:
                break
            if processing_time_q <= job[stage]:
                processing_time_q = job[stage]

            if release_time_q <= job[3]:
                release_time_q = job[3]

            batch_queue.append(job[0])
            current_batch_size += job[4 + stage]

            remaining_jobs = np.delete(remaining_jobs, j, 0)

        return batch_queue, remaining_jobs, processing_time_q, release_time_q

    def construct_batches(self, jobs, k, stage, delta):
        """
        Generates batches from a list of jobs considering batch size and job sizes.

        Args:
        jobs (numpy.ndarray): Array of job data.
        k (int): Maximum size of the batch.
        stage (int): The stage for which the batch is being created.
        delta (int): The space limit for the batch.

        Returns:
        tuple: A tuple containing the list of batches, processing times, and release times.
        """
        if stage == 1:
            jobs_sorted = jobs
        else:
            jobs_sorted = jobs[jobs[:, 3].argsort()]
        batches = []
        processing_times = []
        release_times = []

        while len(jobs_sorted) >= 1:
            new_batch, jobs_sorted, p_q, r_q = self.create_batch_space_limit(jobs_sorted, k, stage, delta)
            batches.append(new_batch)
            processing_times.append(p_q)
            release_times.append(r_q)

        return batches, processing_times, release_times

    def ordered_batches(self, jobs_classified, F, k, stage, delta):
        """
        Organizes batches for each family of jobs.

        Args:
        jobs_classified (list of numpy.ndarray): List of job arrays classified by family.
        F (int): Number of families.
        k (int): Maximum size of the batch.
        stage (int): The stage for which the batch is being created.
        delta (int): The space limit for the batch.

        Returns:
        tuple: A tuple containing the list of batches, processing times, and release times for each family.
        """
        batches = []
        processing_times = []
        release_times = []

        for i in range(F):
            batches_f, pt_f, rt_f = self.construct_batches(jobs_classified[i], k, stage, delta)
            batches.append(batches_f)
            processing_times.append(pt_f)
            release_times.append(rt_f)

        return batches, processing_times, release_times

    def operating_times_batches(self, batches, processing_times):
        """
        Calculates the processing times for batches.

        Args:
        batches (list of list): List of batches, each batch containing job indices.
        processing_times (list of list): List of processing times for each batch.

        Returns:
        numpy.ndarray: Matrix of processing times for each batch.
        """
        max_rows = self.find_max_sublist_length(batches)
        m_processing_times = np.zeros((len(batches), max_rows))

        for j, pt_f in enumerate(processing_times):
            m_processing_times[j, :len(pt_f)] = pt_f

        return m_processing_times

    def find_max_sublist_length(self, lst):
        """
        Finds the length of the longest sublist in a list of lists.

        Args:
        lst (list of list): A list where each element is a sublist.

        Returns:
        int: The length of the longest sublist. Returns 0 if the list is empty or contains no lists.
        """
        if not isinstance(lst, list):
            raise ValueError("Input must be a list")

        return max((len(sublist) for sublist in lst if isinstance(sublist, list)), default=0)

    def schedule_batches_spt(self, release_times, processing_times, num_machines, num_families):
        """
        Schedules batches on machines using the Shortest Processing Time (SPT) rule.

        Args:
        release_times (list of list): Release times for each batch in each family.
        processing_times (list of list): Processing times for each batch in each family.
        num_machines (int): Number of machines available.
        num_families (int): Number of families.

        Returns:
        tuple: A tuple containing the scheduled jobs and a completion vector.
        """
        jobs = []
        for family in range(num_families):
            for job, (release_time, processing_time) in enumerate(zip(release_times[family], processing_times[family])):
                if release_time != 0:
                    jobs.append((family, job, release_time, processing_time))

        machines = [(machine_id, 0, 0) for machine_id in range(1, num_machines + 1)]
        scheduled_jobs = []
        current_time = 0

        while jobs:
            available_jobs = [job for job in jobs if job[2] <= current_time]

            if available_jobs:
                shortest_job = min(available_jobs, key=lambda x: x[3])
                machines.sort(key=lambda x: x[1])
                machine_id, machine_load, last_job_end_time = machines[0]
                start_time = max(last_job_end_time, shortest_job[2])

                completion_time = start_time + shortest_job[3]
                machine_load = completion_time
                machines[0] = (machine_id, machine_load, completion_time)

                assigned_job = {
                    'machine_id': machine_id,
                    'start_time': start_time,
                    'completion_time': completion_time,
                    'family_index': shortest_job[0],
                    'job_index': shortest_job[1]
                }
                jobs.remove(shortest_job)
                scheduled_jobs.append(assigned_job)
                current_time = max(current_time, min(completion_time, machines[1][1]))
            else:
                current_time += 1

        completion_vector = np.array([job['completion_time'] for job in scheduled_jobs]).reshape(-1, 1)
        return scheduled_jobs, completion_vector

    def calculate_times_for_stage(self, batches, scheduled_batches, J):
        """
        Calculates the completion and starting times for each stage.

        Args:
        batches (list of list): List of batches, each batch containing job indices.
        scheduled_batches (list of dict): List of dictionaries containing scheduling information.
        J (int): Total number of jobs.

        Returns:
        tuple: A tuple containing arrays of completion times, starting times, and machine IDs.
        """
        completion_times = np.zeros(J)
        starting_times = np.zeros(J)
        machine_ids = np.zeros(J)

        for batch in scheduled_batches:
            machine_id = batch['machine_id']
            start_time = batch['start_time']
            completion_time = batch['completion_time']
            family_index = batch['family_index']
            job_index = batch['job_index']

            for job in batches[family_index][job_index]:
                completion_times[int(job) - 1] = completion_time
                starting_times[int(job) - 1] = start_time
                machine_ids[int(job) - 1] = machine_id

        return completion_times, starting_times, machine_ids

    def overwrite_release_times(self, jobs, completion_times):
        """
        Overwrites the release times of jobs for the next stage based on completion times.

        Args:
        jobs (numpy.ndarray): Array of job data.
        completion_times (numpy.ndarray): Array of completion times from the previous stage.

        Returns:
        numpy.ndarray: Updated array of job data with new release times.
        """
        jobs_stage_2 = jobs.copy()
        jobs_stage_2[:, 3] = completion_times
        return jobs_stage_2

    def write_solution_file(self, file, scheduled_batches, scheduled_batches_stage_2, starting_times,
                            completion_times_stage_1, machine_ids, starting_times_stage_2, completion_times_stage_2,
                            machine_ids_stage_2, m1, m2, LB):
        """
        Writes the solution of the scheduling heuristic to a file.

        Args:
        file (str): Name of the file to write to.
        scheduled_batches (list): Scheduled batches for stage 1.
        scheduled_batches_stage_2 (list): Scheduled batches for stage 2.
        starting_times, completion_times_stage_1, machine_ids, starting_times_stage_2, completion_times_stage_2, machine_ids_stage_2 (list): Various scheduling parameters.
        m1, m2 (int): Number of machines in stage 1 and 2.
        LB (float): Lower bound of the objective function.

        Returns:
        None: The function writes to a file and does not return anything.
        """
        results_dir = "./Results/"
        os.makedirs(results_dir, exist_ok=True)
        file_path = os.path.join(results_dir, f"solution_GA_{file}")

        try:
            with open(file_path, "w") as f:
                objective = sum(completion_times_stage_2)
                comp1 = [batch['completion_time'] for batch in scheduled_batches]
                comp2 = [batch['completion_time'] for batch in scheduled_batches_stage_2]
                start1 = [batch['start_time'] for batch in scheduled_batches]
                start2 = [batch['start_time'] for batch in scheduled_batches_stage_2]
                gap = round(abs(objective - LB) / abs(objective), 2)
                utilization1 = sum(comp1[i] - start1[i] for i in range(len(comp1))) / ((max(comp1) - min(start1)) * m1)
                utilization2 = sum(comp2[i] - start2[i] for i in range(len(comp2))) / ((max(comp2) - min(start2)) * m2)
                batch1 = len(scheduled_batches)
                batch2 = len(scheduled_batches_stage_2)
                timeInSystem = sum(
                    completion_times_stage_2[i] - self.instance.jobs[i][3] for i in range(len(completion_times_stage_2))) / len(
                    completion_times_stage_2)
                f.write(
                    f"{objective} {gap} {utilization1} {utilization2} {batch1} {batch2} {LB} {max(comp1)} {min(start1)} {max(comp2)} {min(start2)} {timeInSystem}\n")
                # Additional details can be written here if needed
        except IOError as e:
            print(f"Error writing to file: {e}")

    def write_detailed_solution(self, file, scheduled_batches, scheduled_batches_stage_2, starting_times,
                                completion_times_stage_1, machine_ids, starting_times_stage_2, completion_times_stage_2,
                                machine_ids_stage_2, m1, m2, LB):
        """
        Writes a detailed solution of the scheduling heuristic to a file.

        Args:
        file (str): Name of the file to write to.
        scheduled_batches, scheduled_batches_stage_2 (list): Scheduled batches for both stages.
        starting_times, completion_times_stage_1, machine_ids, starting_times_stage_2, completion_times_stage_2, machine_ids_stage_2 (list): Various scheduling parameters.
        m1, m2 (int): Number of machines in stage 1 and 2.
        LB (float): Lower bound of the objective function.

        Returns:
        None: The function writes to a file and does not return anything.
        """
        results_dir = "./Results/"
        os.makedirs(results_dir, exist_ok=True)
        file_path = os.path.join(results_dir, f"detailed_solution_GA_{file}")

        try:
            with open(file_path, "w") as f:
                for i, job in enumerate(self.instance.jobs):
                    f.write(
                        f"{i} {job[0]} {job[1]} {job[2]} {job[3]} {job[4]} {job[5]} {job[6]} {starting_times[i]} {completion_times_stage_1[i]} {starting_times_stage_2[i]} {completion_times_stage_2[i]}\n")
        except IOError as e:
            print(f"Error writing to file: {e}")

    def _old_osx_crossover(self, parent1, parent2):
        crossover_point = np.random.randint(GENES_LENGTH - OSX_SEGMENT_LENGTH)
        child = parent1[crossover_point:crossover_point + OSX_SEGMENT_LENGTH]

        # Initialize a list to keep track of which elements are already in the child
        used_elements = set(child)

        # Iterate through parent2 and add unused elements to the child
        index = 0
        start = []
        while len(start) < crossover_point:
            if parent2[index] not in used_elements:
                start.append(int(parent2[index]))
                used_elements.add(parent2[index])
            index += 1

        while len(used_elements) < GENES_LENGTH:
            if parent2[index] not in used_elements:
                child = np.concatenate((child, [int(parent2[index])]))
                used_elements.add(parent2[index])
            index += 1

        child = np.concatenate((start, child))
        return child

    # One-Point Order Crossover
    def opx_crossover(self, parent1, parent2):
        crossover_point = np.random.randint(OPX_SEGMENT_LENGTH, GENES_LENGTH - OPX_SEGMENT_LENGTH)
        child = parent1[:crossover_point]
        # Initialize a list to keep track of which elements are already in the child
        used_elements = set(child)
        # Iterate through parent2 and add unused elements to the child
        index = 0
        while len(used_elements) < GENES_LENGTH:
            if parent2[index] not in used_elements:
                child = np.concatenate((child, [int(parent2[index])]))
                used_elements.add(parent2[index])
            index += 1

        return child

    # Two-Point Order Crossover
    def tpx_crossover(self, parent1, parent2):
        crossover_point_1 = np.random.randint(GENES_LENGTH - TPX_SEGMENT_LENGTH)
        crossover_point_2 = np.random.randint(crossover_point_1 + TPX_SEGMENT_LENGTH, GENES_LENGTH)
        child = parent1[crossover_point_1:crossover_point_2]

        # Initialize a list to keep track of which elements are already in the child
        used_elements = set(child)

        # Iterate through parent2 and add unused elements to the child
        index = 0
        start = []
        while len(start) < crossover_point_1:
            if parent2[index] not in used_elements:
                start.append(int(parent2[index]))
                used_elements.add(parent2[index])
            index += 1

        while len(used_elements) < GENES_LENGTH:
            if parent2[index] not in used_elements:
                child = np.concatenate((child, [int(parent2[index])]))
                used_elements.add(parent2[index])
            index += 1

        child = np.concatenate((start, child))
        return child

    def crossover(self, parent1, parent2):
        crossover_type = CROSSOVER_TYPE  # np.random.choice(['inversion', 'scramble', 'flip'])
        if crossover_type == 'OPX':
            child = self.opx_crossover(parent1, parent2)
        elif crossover_type == 'TPX':
            child = self.tpx_crossover(parent1, parent2)
        return child

    # Insertion Mutation
    def insert_mutation(self, individual):
        # Select mutation point
        mutation_point = np.random.randint(GENES_LENGTH)
        # Select insertion point
        insertion_point = np.random.randint(GENES_LENGTH)

        # Perform mutation
        gene_to_insert = individual[mutation_point]
        individual = np.delete(individual, mutation_point)
        individual = np.insert(individual, insertion_point, gene_to_insert)

        return individual

    # Flip Mutation
    def flip_mutation(self, individual):
        mutation_indices = np.random.choice(range(GENES_LENGTH), size=2, replace=False)
        _ = individual[mutation_indices[0]]
        individual[mutation_indices[0]] = individual[mutation_indices[1]]
        individual[mutation_indices[1]] = _
        return individual

    # Inversion Mutation
    def inversion_mutation(self, individual):
        mutation_indices = sorted(np.random.choice(range(GENES_LENGTH), size=2, replace=False))
        individual[mutation_indices[0]:mutation_indices[1] + 1] = individual[
                                                                  mutation_indices[0]:mutation_indices[1] + 1][::-1]
        return individual

        # Scramble Mutation

    def scramble_mutation(self, individual):
        mutation_point_1 = np.random.randint(GENES_LENGTH-MUTATION_SCRAMBLE_LENGTH)
        mutation_point_2 = np.random.randint(mutation_point_1 + MUTATION_SCRAMBLE_LENGTH, GENES_LENGTH)
        subsequence = individual[mutation_point_1:mutation_point_2]
        np.random.shuffle(subsequence)
        individual[mutation_point_1:mutation_point_2] = subsequence
        return individual

    # Mutation operation
    def mutate(self, individual):
        if np.random.rand() < MUTATION_RATE:
            mutated_individual = individual.copy()
            # Randomly select mutation type
            mutation_type = MUTATION_TYPE  # np.random.choice(['inversion', 'scramble', 'flip'])
            if mutation_type == 'inversion':
                mutated_individual = self.inversion_mutation(mutated_individual)
            elif mutation_type == 'scramble':
                mutated_individual = self.scramble_mutation(mutated_individual)
            elif mutation_type == 'flip':
                mutated_individual = self.flip_mutation(mutated_individual)
            elif mutation_type == 'insert':
                mutated_individual = self.insert_mutation(mutated_individual)
            return mutated_individual
        return individual
    # Selection operation: Tournament selection
    def tournament_selection(self):
        parents = []
        # Ensure an even number of parents by adjusting if necessary
        if POPULATION_SIZE % 2 != 0:
            parents.append(np.random.randint(POPULATION_SIZE))
        while len(parents) < POPULATION_SIZE:
            tournament_indices = np.random.choice(range(POPULATION_SIZE), size=TOURNAMENT_SIZE, replace=False)
            tournament_scores = [self.fitness_scores[i] for i in tournament_indices]
            winner_index = tournament_indices[np.argmax(tournament_scores)]
            parents.append(winner_index)
        return parents

    # Selection operation: Roulette wheel selection
    def roulette_wheel_selection(self):
        total_fitness = sum(self.fitness_scores)
        probabilities = [score / total_fitness for score in self.fitness_scores]
        parents = np.random.choice(range(POPULATION_SIZE), size=POPULATION_SIZE, p=probabilities)
        return parents

    # Determine selection method dynamically
    def determine_selection_method(self, generation):
        selection_type = SELECTION_TYPE
        if selection_type == 'tournament':
            parents = self.tournament_selection()
            return parents
        elif selection_type == 'roulette':
            parents = self.roulette_wheel_selection()
            return parents

    def hill_climbing_local_search(self, individual):
        # Evaluate the fitness of the current individual
        current_fitness = self.fitness(individual)
        cnt = 0

        # Get the indices of the genes
        gene_indices1 = np.arange(len(individual))
        np.random.shuffle(gene_indices1)
        gene_indices2 = np.arange(len(individual))
        np.random.shuffle(gene_indices2)
        # Iterate over each gene in the individual
        for gene_index in gene_indices1:
            # Try moving the gene to all possible positions within the individual
            for new_position in gene_indices2:
                # Skip if the new position is the same as the current position
                if new_position == gene_index:
                    continue

                cnt += 1
                # Move the gene to the new position
                new_individual = np.copy(individual)
                gene_value = new_individual[gene_index]
                new_individual = np.delete(new_individual, gene_index)
                new_individual = np.insert(new_individual, new_position, gene_value)

                # Evaluate the fitness of the new individual
                new_fitness = self.fitness(new_individual)

                # If the new individual has better fitness, accept the move
                if new_fitness < current_fitness:
                    if DUMMY:
                        print(f"Improvement in local search: {new_fitness - current_fitness}")
                    individual = new_individual
                    current_fitness = new_fitness
                if cnt > LOCAL_SEARCH_MOVES:
                    return individual
        return individual

    def should_restart(self):
        # Check if the algorithm should restart based on a predefined condition
        # For example, lack of improvement in best fitness over multiple generations
        return self.current_restart_attempt < self.max_restart_attempts

    def restart_algorithm(self, elite_population):
        # Reset algorithm state by generating a new population
        self.population = self.regenerate_population(elite_population)
        self.current_restart_attempt += 1

    def regenerate_population(self, elite_population):
        tmp = elite_population.copy()
        for i in elite_population:
            tmp.append(self.insert_mutation(i.copy()))
            tmp.append(self.scramble_mutation(i))
        tmp.append(self.instance.calculate_SPT_jobs())
        tmp.append(self.instance.calculate_ERD_jobs())
        tmp.append(self.instance.calculate_LPT_jobs())
        tmp.extend([np.random.permutation(np.arange(0, GENES_LENGTH, 1)) for _ in
                    range(POPULATION_SIZE - len(tmp))])
        return tmp

    def run(self):
        trggr = False
        previous = np.inf
        self.solution_time = time.time()
        generation = 0
        restart = 0
        while time.time() - self.solution_time < self.instance.TIME_LIMIT:
        #for generation in range(GENERATIONS):
            generation += 1
            restart += 1
            if DUMMY:
                print(f"Generation: {generation}")

            tmp = time.time()
            self.fitness_scores = [self.fitness(individual) for individual in self.population]
            self.fitness_time += time.time() - tmp

            best = np.min(self.fitness_scores)
            if DUMMY:
                print(f"New best fitness: {best}")
                print(f"Gap: {round(abs(best - self.instance.lower_bound) / abs(best), 4)}")
            if best < previous:
                trggr = True
                restart = 1
            self.best_fitness_values.append(best)
            previous = best

            # Adjust elitism every ADJUSTMENT_INTERVAL generations
            if generation % ADJUSTMENT_INTERVAL == -1:
                # Calculate average fitness and adjust elitism percentage
                self.elitism_percentage = min(MAX_ELITISM_PERCENTAGE,
                                              max(MIN_ELITISM_PERCENTAGE,
                                                  (np.min(self.fitness_scores) - self.instance.lower_bound) / np.min(self.fitness_scores)))
                self.elite_size = int(self.elitism_percentage * POPULATION_SIZE)

            # Elitism: Preserve top individuals

            elite_indices = np.argsort(self.fitness_scores)[:self.elite_size]

            elite_population = [self.population[i].copy() for i in elite_indices]

            tmp = time.time()
            if trggr:
                elite_population[0] = self.hill_climbing_local_search(elite_population[0])
                trggr = False
            self.local_search_time += time.time() - tmp

            tmp = time.time()
            # Check if restart condition is met
            if restart % self.restart_interval == 0 or len(set(self.fitness_scores)) < POPULATION_SIZE//2:
                if self.should_restart():
                    if DUMMY:
                        print("Restart the algorithm!")
                    self.restart_algorithm(elite_population)
                    self.fitness_scores = [self.fitness(individual) for individual in self.population]
                    elite_indices = np.argsort(self.fitness_scores)[:self.elite_size]
                    elite_population = [self.population[i].copy() for i in elite_indices]
                    restart = 0
            self.restart_time += time.time() - tmp

            tmp = time.time()
            # Determine selection method for current generation
            # selected_parents_indices = self.determine_selection_method(generation)
            selected_parents_indices = self.roulette_wheel_selection()
            self.selection_time += time.time() - tmp

            tmp = time.time()
            # Crossover
            children = []
            for i in range(0, len(selected_parents_indices), 2):
                parent1_index = selected_parents_indices[i]
                parent2_index = selected_parents_indices[i + 1]
                parent1 = self.population[parent1_index]
                parent2 = self.population[parent2_index]
                if np.random.rand() < CROSSOVER_RATE:
                    # child1 = self.osx_crossover(parent2, parent1)
                    # child2 = self.osx_crossover(parent1, parent2)
                    child1 = self.crossover(parent1, parent2)
                    child2 = self.crossover(parent2, parent1)
                    children.append(child1)
                    children.append(child2)
                else:
                    children.extend([parent1, parent2])
            self.crossover_time += time.time() - tmp

            # Adjust the number of children to maintain a fixed population size
            children = children[:POPULATION_SIZE - self.elite_size]

            tmp = time.time()
            # Mutation
            for i in range(len(children)):
                children[i] = self.mutate(children[i])
            self.mutation_time += time.time() - tmp

            # Replace old population with new population
            self.population = elite_population + children

            # Apply hill climbing local search to a portion of the population
            # for i in range(self.elite_size, POPULATION_SIZE):
            #     if np.random.rand() < LOCAL_SEARCH_RATE:
            #         self.population[i] = self.hill_climbing_local_search(self.population[i])


            # Return best individual
        best_individual_index = np.argmin([self.fitness(individual) for individual in self.population])
        self.solution_time = time.time() - self.solution_time
        if DUMMY:
            self.convergence_analysis()
        self.report()
        self.print_individual(self.population[best_individual_index])
        return best # self.population[best_individual_index]

    def report(self):
        print(f"Solution time: {self.solution_time}")
        print(f"Fitness time: {self.fitness_time}")
        print(f"Selection time: {self.selection_time}")
        print(f"Crossover time: {self.crossover_time}")
        print(f"Mutation time: {self.mutation_time}")
        print(f"Restart time: {self.restart_time}")
        print(f"Local search time: {self.local_search_time}")
        print(f"Total time: {self.fitness_time + self.crossover_time + self.mutation_time + self.local_search_time+ self.selection_time + self.restart_time}")
        print(
            f"Best individual: {self.population[np.argmin([self.fitness(individual) for individual in self.population])]}\n")
        print(
            f"Best fitness: {self.fitness(self.population[np.argmin([self.fitness(individual) for individual in self.population])])}")
        print(f"Gap: {round(abs(self.best_fitness_values[-1] - self.instance.lower_bound) / abs(self.best_fitness_values[-1]), 4)}")


In [ ]:
def main():
    global GENES_LENGTH
    global FOLDER_PATH
    global FILES
    global TXT_FILES
    global TPX_SEGMENT_LENGTH
    global MUTATION_SCRAMBLE_LENGTH
    global MUTATION_INVERSION_LENGTH
    FOLDER_PATH = f"./Data/J_250/" # must be adjusted according to the data
    FILES = os.listdir(FOLDER_PATH)
    TXT_FILES = [f for f in FILES if f.endswith('.txt')]   
    with open(results_file, "w") as f:
        for i in TXT_FILES:
            inst = Instance(i)
            GENES_LENGTH = inst.J
            TPX_SEGMENT_LENGTH = int(GENES_LENGTH // 3)
            MUTATION_SCRAMBLE_LENGTH = int(GENES_LENGTH // 3)
            MUTATION_INVERSION_LENGTH = int(GENES_LENGTH // 3)
            ga = Genetic_Algorithm(inst)
            bst = ga.run()

In [ ]:
if __name__ == "__main__":   
    main()